# Hands on exercise on TAHMO data

Outline
---------
- Dataset description and picture map. 
- Feature extraction step 
- Model construction 


In [1]:
## Map of the stations.. 
from mpl_toolkits.basemap import Basemap
import matplotlib.pylab as plt
%matplotlib inline 
import pandas as pd 
import numpy as np 

In [76]:
import math
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Distance between two geographical location. 
    args:
        lat1: latitude of station 1
        lon1: longtitude of station 1
        lat2: latitude of station 2
        lon2: longtitude of station2
    """
    
    earth_radius = 6371.16
    deg2rad = lambda deg: deg*math.pi/180
    dlat = deg2rad(lat2 - lat1)
    dlon = deg2rad(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(deg2rad(lat1)) * \
    math.cos(deg2rad(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = earth_radius * c #// Distance in km
    return d


<h2>Introduction</h2>

<img src=images/allafrica.png width=75%>

Stations located in Kenya. 

<img src=images/target-stations.png width=75%>

<h3>The TAHMO stations</h3>

<h3>Dataset</h3>

The data is a time series of weather sensor readings, consisting of different physical variables on a regular grid on the Earth, indexed by lon(gitude) and lat(itude) coordinates. The variables we have made available are: 
<ul>
<li>prec --- Percipitation. 
<li>tair --- air temperature. 
<li>relh --- relative humidity. 
<li>wspd --- wind speed. 
<li>wgst --- wind gust. 
<li>pres --- surface pressure.
<li>srad --- Solar radiation. 
</ul>
 
The fields are recorded every 5 minutes for two years from 2016-2017. The dataset is observation averaged on hour scale. 


### Station description

 - from: Station id
 - to: Station id 
 - distance (km): distance between stations.  
 - elevation (m) : Elevation difference between from & to station. 

<h3>The prediction task</h3>
 - Predict whether it rains or not.
 - Predict the amount of rain for the rainy period. 



In [105]:
def nearby_stations(site_code, k=10, radius=100):
    """
    Return k-nearest stations. 
    """
    stations = pd.read_csv("nearest_stations.csv")
    k_nearest = stations[(stations['from'] == site_code) & (stations['distance'] < radius)]
    k_nearest = k_nearest.sort_values(by=['distance', 'elevation'], ascending=True)[0:k]
    
    return k_nearest

In [13]:
dt = pd.read_csv('kenya-tahmo.csv')

In [19]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468196 entries, 0 to 468195
Data columns (total 9 columns):
station     468196 non-null object
datetime    468196 non-null datetime64[ns]
prec        468196 non-null float64
wgst        468196 non-null float64
pres        468196 non-null float64
wspd        467407 non-null float64
srad        467407 non-null float64
tair        467388 non-null float64
relh        460821 non-null float64
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 32.1+ MB


In [16]:
## Convert to datetime 
dt.datetime = pd.to_datetime(dt.datetime)

In [25]:
# Split train and test data. 
X_train = dt[dt.datetime.dt.year==2016]
X_test  = dt[dt.datetime.dt.year==2017]

In [ ]:
# Target stations TA00077, TA00074, TA00025


In [108]:
nearby_stations('TA00074',k=5)

,distance,elevation,from,to
635,19.023836,346,TA00074,TA00056
632,54.625369,-801,TA00074,TA00029
627,56.318508,202,TA00074,TA00024
647,57.152416,-118,TA00074,TA00073
631,68.484114,-294,TA00074,TA00028


In [ ]:
### Feature extraction.

In [ ]:
class FeatureExtractor(object):

    def __init__(self):
        pass

    def transform(self, X_ds):
        X_array = X_ds.groupby([X_ds.datetime.dt.day,'station']).mean()
        return X_array


### Exploratory analysis

In [18]:
dt.head(5)

,station,datetime,prec,wgst,pres,wspd,srad,tair,relh
0,TA00061,2016-01-01 00:00:00,0.00,1.2,80.900000,0.366667,0.000000,13.450000,1.021667
1,TA00061,2016-01-01 01:00:00,0.00,1.4,80.900833,0.433333,0.000000,12.608333,1.006667
2,TA00061,2016-01-01 02:00:00,0.00,1.0,80.943333,0.266667,0.000000,12.091667,0.954167
3,TA00061,2016-01-01 03:00:00,0.00,1.8,80.997500,0.416667,1.373333,12.333333,0.930000
4,TA00061,2016-01-01 04:00:00,0.02,1.2,81.055000,0.375000,26.702500,13.075000,0.905833


### Regression 

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator

class Regressor(BaseEstimator):
    def __init__(self):
        self.clf = LinearRegression()

    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    

1. Hurdel model. Predict whether it rains or not?
2. If it rains how much amount? Possibly use the rainy events data. 

In [92]:
## Load station 
allrain = pd.read_csv('gpm-tahmo-rain.csv')

In [94]:
allrain.datetime = pd.to_datetime(allrain.datetime)

In [95]:
allrain.head(5)

,station,year,month,day,tahmo,gpm,datetime,yday
0,TA00020,2016,1,1,0.0,0.0,2016-01-01,1
1,TA00020,2016,1,2,0.0,0.0,2016-01-02,2
2,TA00020,2016,1,3,0.0,0.0,2016-01-03,3
3,TA00020,2016,1,4,0.0,0.0,2016-01-04,4
4,TA00020,2016,1,5,0.0,0.0,2016-01-05,5


In [99]:
tahmo = allrain[['station','year','datetime','yday','tahmo']]

In [100]:
def k_nearby_features(target_station, k=5, year=2016):
    """
    Extract percipitation from k nearby stations. 
    """
    filter_stn = lambda stn_name, year: tahmo[(tahmo.station==stn_name) & (tahmo.year==year)] 
    t_station = filter_stn(target_station, year)
    k_station = nearby_stations(site_code=target_station, k=k,radius=300).to.tolist()
    X = t_station['tahmo'].as_matrix().reshape([-1,1])
    datetime = tahmo.datetime
    print X.shape
    for stt in k_station:
        tmp = filter_stn(stt,year)
        R = tmp[['tahmo']].as_matrix().reshape([-1,1])
        #print R.shape 
        X = np.hstack([X,R])
    df = pd.concat([datetime,pd.DataFrame(X)], axis=1)
    df.columns = ["datetime",target_station] + k_station
    return df

In [103]:
t_station = k_nearby_features('TA00020')

(366, 1)


In [104]:
t_station.head(5)

,datetime,TA00020,TA00025,TA00057,TA00066,TA00024,TA00067
0,2016-01-01,0.0,0.14,0.04,0.20,0.16,0.00
1,2016-01-02,0.0,0.04,0.00,0.08,0.18,0.00
2,2016-01-03,0.0,0.08,0.04,0.00,0.14,0.12
3,2016-01-04,0.0,3.64,16.49,4.39,7.49,15.84
4,2016-01-05,0.0,0.77,0.95,0.48,0.88,14.56


In [32]:
filter_stn = lambda stn_name, year: tahmo[(tahmo.station==stn_name) & (tahmo.year==year)]
target_station = "TA00021"
year=2016

In [54]:
t_station = filter_stn(target_station, year)
k_station = nearby_stations(site_code=target_station,k=5, radius=500).to.tolist()
  

In [61]:
X = t_station['tahmo'].as_matrix().reshape([-1,1])

In [62]:
X.shape

(366, 1)

In [63]:
for stt in k_station:
    dfs = filter_stn(k_station[1],year)
    R = dfs[['tahmo']].as_matrix()
#R = yX
    R = R.reshape([-1,1])
      #print R.shape
    X = np.hstack([X,R])


In [64]:
X

(366, 6)

In [60]:
X.shape

(366, 2)

In [53]:
R.shape

(366, 1)

In [8]:
## convert to wide format 
tahmo['station_type'] = tahmo['station'].astype(str)

/home/tadeze/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
df = pd.pivot_table(tahmo, index='datetime',columns=['tahmo','station_type'])

In [10]:
df.columns = df.columns.droplevel(0)

In [ ]:
df['payment_type'] = df['payment_type'].astype(str)

df = pd.pivot_table(df, index='date', columns=['vendor', 'payment_type'], aggfunc=max)

#remove top level of multiindex
df.columns = df.columns.droplevel(0)

#reset multicolumns
df.columns = ['_Pay'.join(col).strip() for col in df.columns.values]

print df
            A1_Pay1  A1_Pay2  B1_Pay1  B1_Pay2  C1_Pay1
date                                                   
2015-03-10       50       60      NaN      NaN       65
2015-03-11       45       70      NaN      NaN      NaN
2015-03-12      NaN      NaN       40       45      NaN